In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# metrics
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

#xgb
import xgboost as xgb

In [2]:
tweets = pd.read_csv("data/train_pre_processing.csv")
tweets = tweets.fillna('0')

In [3]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 38 columns):
keyword                 7613 non-null object
location                7613 non-null object
text                    7613 non-null object
keyword_grouped         7613 non-null object
text_contain_keyword    7613 non-null object
total_words             7613 non-null int64
len_text                7613 non-null int64
total_upper_chars       7613 non-null int64
total_numbers_chars     7613 non-null int64
total_special_chars     7613 non-null int64
total_common_chars      7613 non-null int64
contain_question        7613 non-null bool
contain_link            7613 non-null bool
contain_hashtag         7613 non-null bool
contain_upper_words     7613 non-null bool
total_3_words           7613 non-null int64
total_4_words           7613 non-null int64
total_5_words           7613 non-null int64
total_6_words           7613 non-null int64
total_7_words           7613 non-null int64
total_8_word

## Encode: LabelEncoder

In [4]:
encode_columns = ['keyword', 'location', 'text', 'keyword_grouped', 'text_contain_keyword']

encode_tweets = tweets[encode_columns]
encode_tweets = encode_tweets.astype('str')
encode_tweets = encode_tweets.apply(LabelEncoder().fit_transform)
tweets_encode_drop = tweets.drop(encode_columns, axis = 1)
tweets_encode = pd.concat([tweets_encode_drop, encode_tweets], axis = 1)
tweets_encode.drop(axis=1, labels=['target'], inplace=True)

Y = tweets.iloc[:,-1]

In [5]:
seed = 7
test_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(tweets_encode, Y, test_size=0.20, random_state=42)

In [66]:
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42, learning_rate=0.05, 
                             max_depth= 3, n_estimators=400, min_child_weight=5, gamma=0.3, subsample=0.7, 
                             colsample_bytree=0.4, reg_alpha=0.1, seed=123)

xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=0.3, learning_rate=0.05,
       max_delta_step=0, max_depth=3, min_child_weight=5, missing=None,
       n_estimators=400, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=42, reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1,
       seed=123, silent=True, subsample=0.7)

In [70]:
preds = xgb_model.predict(X_test)

In [71]:
roc_auc_score(y_test, preds)

0.7969989034353151

In [ ]:
# 0.7969989034353151